## Installing and Loading Packages

In [49]:
# %pip install selenium
# %pip install webdriver-manager
# %pip install selenium-stealth
# %pip install undetected-chromedriver

In [57]:
import pickle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
from pprint import pprint
import io
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import random
from selenium_stealth import stealth
import undetected_chromedriver as uc


## Web Scraping Tripadvisor Reviews

### Setting up selenium + chrome drivers
- prevent bot detection by: 
    1. cycling through list of user agents

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

def get_free_proxies(driver):
    driver.get('https://sslproxies.org')
    headers = [header.text.strip() for header in driver.find_elements(By.XPATH, '//table/thead/tr/th')]
    rows = driver.find_elements(By.XPATH, '//table/tbody/tr')
    
    proxies = []
    for row in rows:
        proxy_data = dict(zip(headers, [td.text.strip() for td in row.find_elements(By.TAG_NAME, 'td')]))
        proxies.append(proxy_data)
        
    driver.quit()
    return proxies

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

free_proxies = get_free_proxies(driver)
print("Number of free proxies retrieved:", len(free_proxies))
print("5 sample proxy data:")
pprint(free_proxies[:5])

Number of free proxies retrieved: 116
5 sample proxy data:
[{'Anonymity': 'elite proxy',
  'Code': 'US',
  'Country': 'United States',
  'Google': 'no',
  'Https': 'yes',
  'IP Address': '13.56.209.6',
  'Last Checked': '8 secs ago',
  'Port': '8080'},
 {'Anonymity': 'elite proxy',
  'Code': 'CA',
  'Country': 'Canada',
  'Google': 'no',
  'Https': 'yes',
  'IP Address': '67.43.227.230',
  'Last Checked': '8 secs ago',
  'Port': '23685'},
 {'Anonymity': 'elite proxy',
  'Code': 'US',
  'Country': 'United States',
  'Google': 'no',
  'Https': 'yes',
  'IP Address': '165.22.36.164',
  'Last Checked': '8 secs ago',
  'Port': '10001'},
 {'Anonymity': 'elite proxy',
  'Code': 'CA',
  'Country': 'Canada',
  'Google': 'yes',
  'Https': 'yes',
  'IP Address': '72.10.160.94',
  'Last Checked': '8 secs ago',
  'Port': '24271'},
 {'Anonymity': 'anonymous',
  'Code': 'SG',
  'Country': 'Singapore',
  'Google': 'no',
  'Https': 'yes',
  'IP Address': '8.219.97.248',
  'Last Checked': '8 secs ago',


In [18]:
user_agents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Edge/18.0.0.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Edge/18.0.0.0',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/98.0.0.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/98.0.0.0',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Safari/609.1.15',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Safari/609.1.15',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/98.0.0.0',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Edge/18.0.0.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Edge/18.0.0.0',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Safari/609.1.15',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/97.0.0.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/97.0.0.0',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/97.0.0.0',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/17.0.0.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Edge/17.0.0.0',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/17.0.0.0',
]

In [121]:
url = 'https://www.tripadvisor.com/Airlines'
service = ChromeService(ChromeDriverManager().install())
user_agent = random.choice(user_agents)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument("start-maximized")
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)
time.sleep(3)
pickle.dump(driver.get_cookies(), open('cookies.pkl', 'wb'))

In [26]:
def setup_webdriver():
    while True:
        user_agent = random.choice(user_agents)
        print(f"Using user agent: {user_agent}")
        
        proxy_choice = random.choice(free_proxies)
        proxy_ip_port = f"{proxy_choice['IP Address']}:{proxy_choice['Port']}"
        print(f"Using proxy: {proxy_ip_port}")

        service = Service(ChromeDriverManager().install())
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument(f'user-agent={user_agent}')
        chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        chrome_options.add_argument("start-maximized")
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        chrome_options.add_argument(f'--proxy-server={proxy_ip_port}')

        try:
            driver = webdriver.Chrome(service=service, options=chrome_options)
            time.sleep(1)
            driver.quit()
            return driver
        except Exception as e:
            print(f"Error setting up WebDriver: {e}")
            continue

### List of airline reviews in tripadvisor

In [63]:
def scrape_page(driver, csv_writer):
    try:
        airlines = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".prw_rup.prw_airlines_airline_lander_card"))
        )
        for airline in airlines:
            name = airline.find_element(By.CSS_SELECTOR, ".airlineName").text
            review_link_element = airline.find_element(By.CSS_SELECTOR, "a.review_button.ui_button.secondary.small")
            review_link = review_link_element.get_attribute("href")
            num_reviews = airline.find_element(By.CSS_SELECTOR, ".airlineReviews").text
            csv_writer.writerow([name, review_link, num_reviews])

    except (NoSuchElementException, TimeoutException):
        print("Error occurred while scraping.")

In [64]:
def main1():
    driver = setup_webdriver()
    driver.get("https://www.tripadvisor.com/Airlines")
    
    with open('scraped_data.csv', 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Airline Name', 'Review Link', 'Number of Reviews'])

        page_number = 1
        while True:
            print(f"Scraping page {page_number}...")
            scrape_page(driver, csv_writer)

            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "span.nav.next.ui_button.primary"))
                )
                if "disabled" in next_button.get_attribute("class"):
                    print("Reached the last page.")
                    break

                next_button.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
                page_number += 1
                time.sleep(3)

            except (NoSuchElementException, TimeoutException):
                print("Reached the end of the pages or encountered an error.")
                break

    driver.quit()

In [47]:
main1()

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Error occurred while scraping.
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Error occurred while scraping.
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Error occurred while scraping.
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Error occurred while scraping.
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Error occurred while scraping.
Scraping page 41...
Scraping page 42...
Error occurred

### ⚠️ Gather reviews for each airline (deprecated)

In [79]:
def setup_webdriver():
    while True:
        user_agent = random.choice(user_agents)
        print(f"Using user agent: {user_agent}")

        proxy_choice = random.choice(free_proxies)
        proxy_ip_port = f"{proxy_choice['IP Address']}:{proxy_choice['Port']}"
        print(f"Using proxy: {proxy_ip_port}")

        try:
            chrome_options = uc.ChromeOptions()
            chrome_options.add_argument(f'user-agent={user_agent}')
            chrome_options.add_argument('--disable-blink-features=AutomationControlled')
            chrome_options.add_argument("start-maximized")
            chrome_options.add_argument('--disable-dev-shm-usage')
            chrome_options.add_argument('--no-sandbox')
            # chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
            # chrome_options.add_experimental_option('useAutomationExtension', False)
            chrome_options.add_argument(f'--proxy-server={proxy_ip_port}')

            driver = uc.Chrome(options=chrome_options)
            time.sleep(1)
            # driver.quit()
            return driver
        except Exception as e:
            print(f"Error setting up WebDriver: {e}")
            continue

In [80]:
review_elem_selector = 'lgfjP.Gi.z.pBVnE.MD.bZHZM'
rating_elem_selector = 'div.WAllg._T > div.IkECb.f.O > div.Hlmiy.F1 > span'
name_elem_selector = 'div.cRVSd > span > a'
review_title_selector = 'div.WAllg._T > div.KgQgP.MC._S.b.S6.H5._a > a > span > span'
review_tags_1_selector = 'div.WAllg._T > div.IkECb.f.O > div.tNvbV.f.Ng > div:nth-child(1)'
review_tags_2_selector = 'div.WAllg._T > div.IkECb.f.O > div.tNvbV.f.Ng > div:nth-child(2)'
review_tags_3_selector = 'div.WAllg._T > div.IkECb.f.O > div.tNvbV.f.Ng > div:nth-child(3)'
review_text_selector = 'div.WAllg._T > div.vTVDc > div._T.FKffI.bmUTE > div.fIrGe._T > span'
review_travel_date_selector = 'div.WAllg._T > div.vTVDc > div.Pq > span'

In [86]:
def scrape_reviews(driver, url):
    driver.get(url)
    print("the url has been opened")
    time.sleep(5)

    reviews = []
    
    review_elements = driver.find_elements(By.CLASS_NAME, review_elem_selector)
    print("review elements found")
    print(review_elements)
    for review_element in review_elements:
        try:
            rating = review_element.find_element(By.CSS_SELECTOR, rating_elem_selector).text
            name = review_element.find_element(By.CSS_SELECTOR, name_elem_selector).text
            review_title = review_element.find_element(By.CSS_SELECTOR, review_title_selector).text
            tags_1 = review_element.find_element(By.CSS_SELECTOR, review_tags_1_selector).text
            tags_2 = review_element.find_element(By.CSS_SELECTOR, review_tags_2_selector).text
            tags_3 = review_element.find_element(By.CSS_SELECTOR, review_tags_3_selector).text
            review_text = review_element.find_element(By.CSS_SELECTOR, review_text_selector).text
            travel_date = review_element.find_element(By.CSS_SELECTOR, review_travel_date_selector).text

            reviews.append({
                "rating": rating,
                "name": name,
                "review_title": review_title,
                "tags_1": tags_1,
                "tags_2": tags_2,
                "tags_3": tags_3,
                "review_text": review_text,
                "travel_date": travel_date
            })
        except Exception as e:
            print(f"Error: {e}")

    return reviews

In [87]:

def main2(url):
    driver = setup_webdriver()
    print(url)
    
    try:
        print("started scraping reviews")
        reviews = scrape_reviews(driver, url)
        with open('reviews.csv', 'w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames = ["rating", "name", "review_title", "tags", "tags_1", "tags_2", "tags_3", "review_text", "travel_date"]
)
            writer.writeheader()
            writer.writerows(reviews)

    finally:
        print("failed to open")
        driver.quit()

In [88]:
test_url = "https://www.tripadvisor.com/Airline_Review-d17550095-Reviews-Advanced-Air"
main2(test_url)

Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36
Using proxy: 167.172.238.15:10004
https://www.tripadvisor.com/Airline_Review-d17550095-Reviews-Advanced-Air
started scraping reviews
the url has been opened
review elements found
[]
failed to open


In [78]:
# test_url = "https://www.tripadvisor.com/Airline_Review-d17550095-Reviews-Advanced-Air"
# service = ChromeService(ChromeDriverManager().install())
# user_agent = random.choice(user_agents)
# chrome_options = webdriver.ChromeOptions()
# proxy_choice = random.choice(free_proxies)
# proxy_ip_port = f"{proxy_choice['IP Address']}:{proxy_choice['Port']}"
# print(f"Using proxy: {proxy_ip_port}")
# chrome_options.add_argument(f'--proxy-server={proxy_ip_port}')
# chrome_options.add_argument(f'user-agent={user_agent}')
# chrome_options.add_argument('--disable-blink-features=AutomationControlled')
# chrome_options.add_argument("start-maximized")
# chrome_options.add_argument('--disable-dev-shm-usage')
# # chrome_options.add_argument('--no-sandbox')
# chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
# chrome_options.add_experimental_option('useAutomationExtension', False)
# driver = webdriver.Chrome(service=service, options=chrome_options)
# driver.get(test_url)
# time.sleep(3)
# # main2(test_url)
# driver.quit()

### Gather each airline reviews 